In [1]:
import networkx as nx

from rdflib import Graph, URIRef, Literal, Namespace
from pathlib import Path
from operator import itemgetter
from rdflib.namespace import XSD, RDF, RDFS, SKOS

from rdflib.extras.external_graph_libs import *

Visualizing the graph may improve our understanding of the data in the graph. The IReC GitHub repository contains some instructions for visualizing the graph with GraphDB. The `unfeatured_graph.ttl` file probably contains the most appropriate level of information for visualizing our graph. 

In this notebook, we would like to analyse the graph to identify potential concepts. The idea being that they could be added to a manually curated AEC conceptualization. For analysis, we'll use the `featured_graph.ttl` file.

We'll look at some basic graph analysis approaches:
* **Paths** we could be interested in, e.g.;
  * X constitutes Y constitutes Z etc
  
* Finding **central nodes** based, e.g.;
  * which nodes are likely common high-level classes, e.g., materials, systems, etc. (would we want to reweigh phrasese that contain such words?)
  * features that we could consider include the degree (number of edges for a node), betweennes (number of shortest paths that pass through the node), and 'page rank' (importance of node based on importance of related nodes)
      * concept-concept shortest paths may identify related terms, e.g., determine spans that connect the concepts `fire doorset` and `protected corridor`
      * betweennes centrality may suggest high level concepts, e.g., `materials`, `systems` and so on
      * page rank; which spans are connected to the most influential spans (ones with high betweennes or connected direclty to concepts)
* **Community detection**; find groups of closely connected nodes, e.g., we'd like to find all nodes that are directly related to spans revolving around materials. Approaches to use include Label propagation and Unfolding (Louvain) to compute a bottom-up grouping of connected nodes.
  
For now, we'll avoid looking at pattern matching through worst case optimal join (WCOJ) algorithms.


Add to requirements.txt:
* scipy==1.8
* networkx==3.0

In [2]:
graph_output_fp = Path.cwd().joinpath("data", "graph_output")

In [21]:
IREC_spans_URL = "https://spans.irec.org/"
IREC_concepts_URL = "https://concepts.irec.org/"

### Load Graph

In [57]:
irec_graph = Graph()
graph_data_fp = Path.cwd().joinpath("data", "graph_data")
irec_graph.parse(graph_data_fp.joinpath("IREC.rdf"))
irec_graph.parse(graph_output_fp.joinpath("unfeatured_graph.ttl"))
UG = rdflib_to_networkx_graph(irec_graph)

In [60]:
# convert to NetworkX graph
irec_graph.parse(graph_output_fp.joinpath("featured_graph.ttl"))
FG = rdflib_to_networkx_graph(irec_graph)
# G = rdflib_to_networkx_digraph(irec_graph)

In [69]:
test_query = """
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix irec: <https://schema.irec.org/#>

SELECT DISTINCT ?aname ?bname
WHERE {
    ?a irec:constitutes ?b .
    ?a rdfs:label ?aname .
    ?b rdfs:label ?bname . 
}
"""
res = irec_graph.query(test_query)

In [70]:
for row in res:
    if row.aname != row.bname:
        print(f"{row.aname} <-> {row.bname}")

www iats - uk org / iats - member - list / > member
structural member in architecture > member
member states > member
member state > member
horizontal member > member
sides member > member
horizontal structural member > member
vertical structural member > member
structural member > member
a horizontal member > member
a member > member
A member > member
the member > member
an independent stairway SECTION AA SECTION BB A B B 1 8m > AA
AA > an independent stairway SECTION AA SECTION BB A B B 1 8m
BB > an independent stairway SECTION AA SECTION BB A B B 1 8m
independent > an independent stairway SECTION AA SECTION BB A B B 1 8m
stairway > an independent stairway SECTION AA SECTION BB A B B 1 8m
A > an independent stairway SECTION AA SECTION BB A B B 1 8m
an A > an independent stairway SECTION AA SECTION BB A B B 1 8m
measures > Alternative caller - identification measures
Alternative caller - identification measures > measures
Article 7  health monitoring measures > measures
measures relia

MB kg ) Mass per unit area > MB
MB > MB kg ) Mass per unit area
Mass per unit area > MB kg ) Mass per unit area
Mass per > MB kg ) Mass per unit area
unit > MB kg ) Mass per unit area
area > MB kg ) Mass per unit area
unit area > MB kg ) Mass per unit area
MCRMA Technical Paper 14 Guidance > MCRMA
MCRMA > MCRMA Technical Paper 14 Guidance
Guidance > MCRMA Technical Paper 14 Guidance
Measuring Association > Measuring
Measuring > Measuring Association
Association > Measuring Association
O > Factor O
Factor O > O
OFTEC Oil Fired Appliance Standard > OFTEC
OFTEC > OFTEC Oil Fired Appliance Standard
Appliance > OFTEC Oil Fired Appliance Standard
Standard > OFTEC Oil Fired Appliance Standard
OJ C 96 1 4 1996 > 1996
The Pipelines Safety Regulations 1996 ii > 1996
BS 6465 - 2 : 1996 Sanitary installations > 1996
BS 853 - 1 : 1996 Specification > 1996
Cavity Barriers Pipelines Safety Regulations 1996 > 1996
SI 1996 / 825 > 1996
the Pipelines Safety Regulations 1996 > 1996
March 1996 > 1996
BS E

commercial use, shop an commercial use > commercial
commercial off the shelf > commercial
commercial A retail trade > commercial
commercial : without sprinkler 90 min 60 min 60 min 60 min 90 min Not system > commercial
commercial organisations > commercial
residential and commercial building > commercial
commercial waste > commercial
commercial premises > commercial
shop commercial > commercial
commercial platform > commercial
commercial space > commercial
commercial area > commercial
commercial buildings > commercial
commercial building > commercial
commercial use, shop an commercial use > shop
shop bsigroup com / > shop
shop commercial > shop
commercial use, shop an commercial use > shop commercial
commercial > shop commercial
shop > shop commercial
courtyard > architectural feature : courtyard
architectural > architectural feature : courtyard
feature > architectural feature : courtyard
architectural feature : courtyard > courtyard
day - to - day activities > activities
bureaucratic 

spontaneous combustion > spontaneous
spontaneous > spontaneous combustion
combustion > spontaneous combustion
technical specification TS 008 : 2012 > 2012
No 722 / 2012 y / n ) > 2012
BSI Published Document PD 6693 - 1 : 2012 > 2012
SAP 2012 Appendix R > 2012
SAP 2012 Table 6e > 2012
ISO 15189 : 2012 > 2012
October 2012 > 2012
SAP 2012 > 2012
March 2012 > 2012
SI 2012 / 3118 > 2012
PAS 24 : 2012 > 2012
Fire and Escape Doors [ 2012 ] Approach > 2012
BS EN 997 : 2012 > 2012
Regulations 2012 > 2012
technical specification TS 008 : 2012 > technical
A favourable technical assessment > technical
an industry technical body > technical
european technical assessments > technical
technical condition > technical
industry technical body > technical
technical methods > technical
technical publications > technical
technical system > technical
technical > technical specification TS 008 : 2012
2012 > technical specification TS 008 : 2012
E > the E 20 Sa
thoroughfare > a pedestrian thoroughfare
pedestr

BS > BS EN 13501 - 3
BS EN > BS EN 13501 - 3
Cavity Barriers Pipelines Safety Regulations 1996 > Barriers
Barriers > Cavity Barriers Pipelines Safety Regulations 1996
1996 > Cavity Barriers Pipelines Safety Regulations 1996
Cavity > Cavity Barriers Pipelines Safety Regulations 1996
Safety > Cavity Barriers Pipelines Safety Regulations 1996
Safety ) > Cavity Barriers Pipelines Safety Regulations 1996
Regulations 1996 > Cavity Barriers Pipelines Safety Regulations 1996
Regulations > Cavity Barriers Pipelines Safety Regulations 1996
Design  s New Homes 2014 > 2014
Second Edition [ 2014 > 2014
Design  s New Homes 2014 > Design
www firecode org uk Fire Safe Design : A New Hardware > Design
BS EN 1994 - 1 - 1 : 2004 Eurocode 4 : Design > Design
Eurocode 8 : Design of structures > Design
Section 2 Design standards > Design
Design of sprinkler systems E3 > Design
the Design of Healthcare Tests [ 2000 ] Premises > Design
www hmppsintranet org uk / uploads / Building Bulletin BB ) 100 : Design f

...

the building > the building compartment
compartment > the compartment wall
the compartment wall floor > the compartment wall
wall > the compartment wall
compartment wall  > the compartment wall
compartment wall > the compartment wall
the wall > the compartment wall
compartment > the compartment wall floor
floor > the compartment wall floor
wall > the compartment wall floor
compartment floor > the compartment wall floor
compartment wall  > the compartment wall floor
compartment wall > the compartment wall floor
wall floor > the compartment wall floor
the compartment wall > the compartment wall floor
the wall floor > the compartment wall floor
the floor > the compartment wall floor
the wall > the compartment wall floor
construction > Fire resisting construction
Fire resisting > Fire resisting construction
construction > building construction
building construction term > building construction
building > building construction
construction > composite construction material
material > compos

the rooms > the source rooms
area > sales area c
Table 9 1 > Table 9
Table 9 > Table 9 1
Table 1 > Table 9 1
space > conference space
surface > Inner surface
surface > The exposed surface
The surface > The exposed surface
surface > mm surface
surface > s surface
surface > the continents surface
the surface > the continents surface
surface > the effectiveness surface
the surface > the effectiveness surface
surface > the floor surface
floor > the floor surface
the surface > the floor surface
the floor > the floor surface
surface > the hob surface
the surface > the hob surface
surface > the reference surface
the surface > the reference surface
surface > the surface area
area > the surface area
surface area > the surface area
the surface > the surface area
the area > the surface area
surface > the wall surface
wall > the wall surface
the surface > the wall surface
the wall > the wall surface
space > turning space
Purge ventilation > Purge ventilation room
room > Purge ventilation room
vent

wall > Decorative wall coverings
wall coverings > Decorative wall coverings
Diagram > Diagram 34
Diagram > Diagram 5 2a
Diagram 5 > Diagram 5 2a
Diagram > Diagram D4
Table 6 > Table 6 2
Table 2 > Table 6 2
wall > wall in Kilsby
wall in > wall in Kilsby
water > water proofing
water > water softeners
Diagram 11 5 > Diagram 11
Diagram 3 11 > Diagram 11
Diagram > Diagram 11
Diagram 11 > Diagram 11 5
Diagram > Diagram 11 5
Diagram 5 > Diagram 11 5
Diagram 13 5 > Diagram 13
Diagram 2 13 > Diagram 13
Diagram 13 1 > Diagram 13
Diagram > Diagram 13
Diagram 13 > Diagram 13 5
Diagram > Diagram 13 5
Diagram 5 > Diagram 13 5
Diagram > Diagram 26
Regulations > Regulations 2004
Regulations > the Regulations 2011
roof > roof valley
water > water located
paragraph 2 25 > 2 25
2 25 > paragraph 2 25
Diagram 16 1 > Diagram 16
Diagram > Diagram 16
Diagram 16 > Diagram 16 1
Diagram > Diagram 16 1
Diagram 1 > Diagram 16 1
Diagram > Diagram 21
door > patio door
door > revolving door
systems > Chimney systems


paragraph 3 15 > paragraph 15 3
paragraph 15 3 > paragraph 3 15
Diagram 1 1 > Diagram 1
Diagram 17 1 > Diagram 1
Diagram 16 1 > Diagram 1
Diagram 13 1 > Diagram 1
Diagram 14 1 > Diagram 1
Diagram 18 1 > Diagram 1
Diagram 15 1 > Diagram 1
Diagram 12 1 > Diagram 1
Diagram 9 1 > Diagram 1
Diagram 8 1 > Diagram 1
Diagram 7 1 > Diagram 1
Diagram 1 6 > Diagram 1
Diagram 5 1 > Diagram 1
Diagram 1 4 > Diagram 1
Diagram 6 1 > Diagram 1
Diagram > Diagram 1
Diagram 4 1 > Diagram 1
Diagram 1 3 > Diagram 1
Diagram 3 1 > Diagram 1
Diagram 2 1 > Diagram 1
Diagram 1 2 > Diagram 1
building > Most building work
building work > Most building work
the structural design > structural design
structural design > the structural design
Cavity wall ties > wall ties
wall > wall ties
Diagram 7 > Diagram 3 7
Diagram > Diagram 3 7
Diagram 3 > Diagram 3 7
Diagram 3 7 > Diagram 7
Diagram 2 7 > Diagram 7
Diagram 7 1 > Diagram 7
Diagram > Diagram 7
Diagram > Diagram 30
water > water in
Diagram 7 > Diagram 2 7
Diagram > 

the water > the water table
fire > fire spreads
fire > fire spreading
wall > buttressing wall
the buttressing wall > buttressing wall
buttressing wall > the buttressing wall
wall > the buttressing wall
the wall > the buttressing wall
wall > curtain wall
wall > return wall
the return wall > return wall
return wall > the return wall
wall > the return wall
the wall > the return wall
wall > gable wall
wall > Every wall
An external wall > An wall
wall > An wall
wall > basin wall
building > building societies
wall > party wall
wall > wall tie
building > single-storey building
building > temporary building
building > multi-storey building
building envelope  > the building envelope
building envelope > the building envelope
building > the building envelope
the envelope > the building envelope
the building > the building envelope
building owner > The building owner
building > The building owner
The owner > The building owner
The building > The building owner
building > building owner
The buildin

### Identify Paths

In [40]:
UG.edge_subgraph('')

TypeError: edge_subgraph() missing 1 required positional argument: 'edges'

### Connectedness

In [10]:
connected_components = list(nx.connected_components(G))
len(connected_components)

240

In [11]:
span_degree_dict = dict(G.degree((g for g in G.nodes() if str(g).startswith(IREC_spans_URL))))

In [12]:
spans_sorted_by_num_edges = sorted(span_degree_dict.items(), key=itemgetter(1), reverse=True)
print("Top 20 spans by degree:")
for idx, span_node in enumerate(spans_sorted_by_num_edges[1:21]):
    [print(f"{idx + 1}: {s_triple[2]} [{span_node[1]}]") for s_triple in irec_graph.triples((span_node[0], RDFS.label, None))]

Top 20 spans by degree:
1: house [229]
2: a building [103]
3: building [65]
4: water [61]
5: Hotel [55]
6: pipe [53]
7: Chimneys [50]
8: wall [48]
9: room [45]
10: area [45]
11: extension [44]
12: maintenance [43]
13: ceiling [42]
14: stone [41]
15: people [41]
16: ventilation [41]
17: material [41]
18: the building [40]
19: gallery [38]
20: fire doorset [38]


In [13]:
concept_degree_dict = dict(G.degree((g for g in G.nodes() if str(g).startswith(IREC_concepts_URL))))

In [14]:
concepts_sorted_by_num_edges = sorted(concept_degree_dict.items(), key=itemgetter(1), reverse=True)
print("Top 20 concepts by degree:")
for idx, d in enumerate(concepts_sorted_by_num_edges[:20]):
    # note that the list comprehension here exists for yielding from the generator
    # also the only reason we are skipping the schemeUID is because it doesn't have a prefLabel
    [print(f"{idx}: {c_triple[2]} [{d[1]}]") for c_triple in irec_graph.triples((d[0], SKOS.prefLabel, None))]

Top 20 concepts by degree:
1: fire doorset [36]
2: display window [34]
3: total useful floor area [33]
4: wet room [32]
5: key elements [31]
6: element of structure [31]
7: comité européen de normalisation [30]
8: commissioning [29]
9: unprotected area  [29]
10: specialist process lighting [27]
11: softened wholesome water [26]
12: dwelling [26]
13: unvented hot water storage system [26]
14: room [24]
15: external wall [24]
16: building [23]
17: pipe [23]
18: united kingdom accreditation service [23]
19: surface water activity [23]


### Node centrality

In graph theory, a clustering coefficient is a measure of the degree to which nodes in a graph tend to cluster together. Evidence suggests that in most real-world networks, and in particular social networks, nodes tend to create tightly knit groups characterized by a relatively high density of ties; this likelihood tends to be greater than the average probability of a tie randomly established between two nodes (Wikipedia)

In [ ]:
clustering = nx.clustering(G)
len(clustering)

In [ ]:
highly_connected_nodes = [n for n, clustering_coefficient in clustering.items() if clustering_coefficient == 0]

In [ ]:
len(highly_connected_nodes)

In [ ]:
print("20 example outlier spans")
for idx, span_node in enumerate(highly_connected_nodes[1:21]):
    [print(f"{idx + 1}: {s_triple[2]}") for s_triple in irec_graph.triples((span_node, RDFS.label, None))]

In [23]:
louvain = nx.algorithms.community.louvain.louvain_communities(G)

In [24]:
len(louvain)

248

In [25]:
for idx, community in enumerate(louvain):
    print(f"Community [{idx}] of size {len(community)}, 20 spans by degree:")
    span_degree_dict = dict(G.degree((g for g in community if str(g).startswith(IREC_spans_URL))))
    spans_sorted_by_num_edges = sorted(span_degree_dict.items(), key=itemgetter(1), reverse=True)
    for idx, span_node in enumerate(spans_sorted_by_num_edges[1:21]):
        [print(f"{idx + 1}: {s_triple[2]} [{span_node[1]}]") for s_triple in irec_graph.triples((span_node[0], RDFS.label, None))]

Community [0] of size 1775 op 20 spans by degree:
1: heating system [223]
2: ventilation systems [221]
3: ventilation system [220]
4: Heating systems [207]
5: water [205]
6: storage system [201]
7: surface water [200]
8: Protection systems [197]
9: Lighting systems [193]
10: pipe systems [190]
11: gas systems [185]
12: water level [176]
13: ventilation opening [172]
14: Ventilation openings [170]
15: solar system [169]
16: mechanical systems [169]
17: alarm systems [167]
18: ventilation openings [167]
19: water storage [166]
20: space heating [166]
Community [1] of size 2560 op 20 spans by degree:
1: The building [1446]
2: the wall [1417]
3: the walls [1411]
4: the Wall [1407]
5: the area [1383]
6: the floor [1383]
7: the system [1367]
8: The system [1361]
9: the roof [1347]
10: the door [1345]
11: the water [1344]
12: The doors [1343]
13: the doors [1339]
14: The door [1337]
15: the rooms [1335]
16: the roofing [1333]
17: the ventilation [1325]
18: the space [1323]
19: the ventilator 

{rdflib.term.URIRef('https://spans.irec.org/#convection'),
 rdflib.term.URIRef('https://spans.irec.org/#frail'),
 rdflib.term.Literal('Rainwater drainage systems', lang='en'),
 rdflib.term.Literal('metal', lang='en'),
 rdflib.term.URIRef('https://spans.irec.org/#rh-negative'),
 rdflib.term.URIRef('https://spans.irec.org/#trussed%20beam'),
 rdflib.term.URIRef('https://spans.irec.org/#museum'),
 rdflib.term.Literal('incompetent', lang='en'),
 rdflib.term.Literal('gradients', lang='en'),
 rdflib.term.URIRef('https://www.wikidata.org/wiki/#Q113459391'),
 rdflib.term.URIRef('https://spans.irec.org/#flexible%20membrane'),
 rdflib.term.Literal('balcony', lang='en'),
 rdflib.term.URIRef('https://spans.irec.org/#effortless'),
 rdflib.term.Literal('tonal', lang='en'),
 rdflib.term.URIRef('https://spans.irec.org/#Commission%20Decision%202000%20/%20553%20/%20EC'),
 rdflib.term.Literal('a variety of sound, usually meaning any unwanted sound', lang='en'),
 rdflib.term.Literal('Offices', lang='en'),


In [7]:
L = nx.laplacian_matrix(G)

In [8]:
L

<35592x35592 sparse array of type '<class 'numpy.int64'>'
	with 224167 stored elements in Compressed Sparse Row format>